<a href="https://colab.research.google.com/github/neurologic/NeurophysiologyModules/blob/main/EMG_AntagonisticPairs_WithKinematics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a id="one"></a>

# By collecting data for this lab and thoughtfully working through the analysis, you will learn more about what it means for neural activity to be coordinated, and how activity in the motor system relates to movement in the world. 
Some of the terms and analytic techniques that you will be using:
- correlation
- EMG
- amplitude envelope
- dimension/dimensionality
- variance
- orthogonal
- kinematics

### Throughout the notebook, you will plot raw data and results of analysis.
- You can interact with the plots by zooming in and panning. <br>
- You can make the plot bigger or smaller by dragging its bottom right corner (gray triangle). Note that when it gets smaller the axis labels might disappear.
- You can save the current plot view at any time by hitting the "save" icon - it will save to your Downloads folder. <br>

## Part I. Initialize notebook.
As you now know, Python's Jupyter Lab notebooks are a way to combine executable code, code outputs, and text into one connected file. They run using a <b>'kernel'</b>, which is the thing that executes your code. It is what connects the notebook (as you see it) with the part of your computer, or the DataHub computers, that runs code. When you 'run'/'execute' code cells, the kernel does what the code tells it to, for example: importing software packages, importing data, plotting data, calculating correlation between signals, etc. Instructions to 'run' code cells are contained in green boxes within these 'markdown' (text) cells. Make sure to execute the cells in order throughout the notebook without skipping over cells. 

<div class="alert alert-success"><b>Task:</b> Run the cell below to import necessary packages and set up the coding environment.</div>

In [ ]:
# No need to edit anything in this code cell
#################################

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import csv
from scipy.signal import hilbert,medfilt,resample
from sklearn.decomposition import PCA
import scipy
import seaborn as sns
import datetime
import matplotlib.animation as animation
%matplotlib widget
# https://towardsdatascience.com/how-to-produce-interactive-matplotlib-plots-in-jupyter-environment-1e4329d71651

fs = 100
fs_vid = 30

def get_bodypart_speed(df_vid,d,new_name):
    dd = np.diff(df_vid[d].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

def get_bodypart_speed_absolute(df_vid,x,y,new_name):
    dx = np.diff(df_vid[x].values)
    dy = np.diff(df_vid[y].values)
    dpos = np.sqrt(dx**2 + dy**2)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dpos/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

def get_bone_speed(df_vid,bone,new_name):
    dd = np.diff(df_vid[bone].values)
    fs_ = 1/np.mean(np.diff(df_vid['time']))
    speed = np.concatenate([[0],dd/(1/fs_)])
    filtert = int(0.15*fs_)
    speed = scipy.ndimage.gaussian_filter(speed,filtert)
    df_vid[new_name] = (speed - np.mean(speed))/np.std(speed)
    return df_vid

### Edit the code cell below with the appropriate information, then play/execute the cell
> all file names and paths need to be in quotations, ***if you are on a windows operating system computer, you need an "r" before the first quote of a filepath***
- **folderpath** is the path to your muscle data (.csv file) that has simultaneous recording of two muscles as well as the tracking data (.h5 files) from the video analysis. 
- **emg_file** = the ".csv" data file that has simultaneous recording of two muscles.
- **joints_file** = the "_filtered.h5" file from video tracking analysis
- **bones_file** the "_skeleton.h5" file from video tracking analysis
- **videotimestamps_file** = the ".csv" file that has timestamps from the video camera input during recording
- **channels** = which arduino channels provided your amplified muscle data to bonsai.

<div class="alert alert-success"><b>Task:</b> Running the cell below (after entering all of the filepaths and filenames requested) will tell the computer where to look for your data.</div>

In [ ]:
folderpath = "/Users/kperks/mnt/OneDrive - wesleyan.edu/Teaching/Neurophysiology/Data/EMG_Tracking/20211031/"

emg_file = "analoginput_zip2021-10-31T13_05_29.csv"
joints_file = 'video2021-10-31T13_05_30DLC_resnet101_EMG_Tracking_ZooInitializedNov3shuffle1_20000_filtered.h5'
bones_file = 'video2021-10-31T13_05_30DLC_resnet101_EMG_Tracking_ZooInitializedNov3shuffle1_20000_skeleton.h5'
videotimestamps_file = "video_timestamps2021-10-31T13_05_29.csv"

channels = ['1','5'] #channels recorded


# No need to edit anything below this line
#################################

emg_path = Path(folderpath) / emg_file
joints_path = Path(folderpath) / joints_file
bones_path = Path(folderpath) / bones_file
video_path = Path(folderpath) / videotimestamps_file

print('completed at ' + str(datetime.datetime.now()))

<a id="two"></a>
## Part II. Import raw EMG data and process the signal.
<div class="alert alert-success"><b>Task:</b> Running the cell below actually imports the raw EMG data and formats it in a way that can be plotted.</div>

In [ ]:
#%% import data
df_analog_ = pd.read_csv(emg_path.expanduser().resolve(), sep=',')

# rename headers as input channels
for i,h in enumerate(list(df_analog_.columns)[:-1]):
    df_analog_.rename(columns={h : str(i)},inplace=True)
    
# the last column is timestamps    
df_analog_.rename(columns={list(df_analog_.columns)[-1] : 'time'},inplace=True)

# reformat 'Time' to start at t0 = 0 and be units of seconds
# xtime = (df_analog['Time']-df_analog['Time'].iloc[0])/1000
# reformat 'Time' to be units of seconds
xtime = (df_analog_['time'])/1000
df_analog_['time'] = xtime
x = np.linspace(df_analog_['time'].iloc[0],df_analog_['time'].iloc[-1],
                int((df_analog_['time'].iloc[-1]-df_analog_['time'].iloc[0])*fs))
df_mat = [df_analog_.loc[np.max(df_analog_[df_analog_['time']<=x_].index)].values for x_ in x]
df_analog = pd.DataFrame(df_mat,columns = df_analog_.columns)
df_analog = df_analog.loc[:,channels]
df_analog['time']=x-x[0]

### Plot the raw EMG signal from each muscle
> The amplitude units are in Volts, but the absolute amplitude of the signals are not important for the analyses of this data that we will do for this lab. In the next step, you will be obtaining the "amplitude envelope" of this raw signal, which discards absolute Voltage information anyway. <br>
<br>
> The "baseline" of each channels is offset from each other so that they are not visually overlaid - the baseline voltage of the recording on each channel was actually zero. 

<div class="alert alert-success"><b>Task:</b> Running the cell below plots the raw EMG signal across time from each channel recorded.</div>

In [ ]:
# No need to edit this code cell
################################
hfig,axs = plt.subplots(1)
axs.set_xlabel('seconds')
axs.set_ylabel('amplitude')
axs.set_yticklabels([])
axs.plot(df_analog['time'],df_analog[channels[0]].values,linewidth = 2,color = 'blue')
axs.plot(df_analog['time'],df_analog[channels[1]].values + np.median(df_analog[channels[0]].values),linewidth = 2,color = 'orange')


### Look at the raw data that you just plotted. 
#### Pick a time window to analyze that has decently stereotyped patterned behavior and enter it in the code cell below
- **start** = the time in your recording when stereotyped patterned behavior became clear.
- **stop** = the time in your recording when stereotyped patterned behavior stopped.

<div class="alert alert-success"><b>Task:</b> Edit the code cell below to define the region of your recording used for analysis. Then run the cell.</div>

In [ ]:
start = 11.79
stop = 47.06


# No need to edit below this line
################################
print('all set - analysis domain defined')

### Process the raw EMG signal into a rectified, smoothed "amplitude envelope"
> This is a type of *signal processing*. You have already encountered this type of signal processing when you analyzed the extracellular cockroach data that you collected earlier this semester. Remind yourself of what this does to the raw signal that you recorded. The result here is a processed signal that we will refer to as the *amplitude envelope* of the EMG activity. 

<div class="alert alert-success"><b>Task:</b> Run the cell below to process the raw EMG signal and get its "envelope". You will also get a plot of the EMG envelope. You do not need to edit anything in the code cell.</div>

In [ ]:
# No need to edit this code cell
################################

# Use rectfication and gaussian smoothing on EMG to get mean-centered rate
df_rate = pd.DataFrame({})
filtert = int(0.05*fs)
for h in list(df_analog.columns)[:-1]: # rename headers as input channels
    y = df_analog[h] - np.mean(df_analog[h])
    y = np.abs(y) #takes the absolute value of 
    y = scipy.ndimage.gaussian_filter(y,filtert)
    df_rate[h] = y
# df_rate = df_rate.subtract(df_rate.mean())
df_rate =(df_rate - df_rate.mean()) / df_rate.std()
df_rate['time']=df_analog['time']
df_rate = df_rate[((df_rate['time']>start) & (df_rate['time']<stop))]

hfig,ax = plt.subplots(1)
ax.set_xlabel('seconds')
ax.set_ylabel('amplitude')
ax.set_yticklabels([])
ax.plot(df_rate['time'],df_rate[channels[0]].values,linewidth = 2,color = 'blue')
ax.plot(df_rate['time'],df_rate[channels[1]].values,linewidth = 2,color = 'orange')

### Think about how you would qualitatively describe the relationship between the two muscle signals. As part of your answer, include a statement about correlation. 
> Essentially, correlation is the measure of how two or more variables are related to one another. https://en.wikipedia.org/wiki/Correlation

<div class="alert alert-success"><b>Task:</b> Running the cell below creates two figures. One is a scatter plot of the EMG envelope of one muscle against the other muscle. Think about how this is a way to visualize *correlation*. The second is a *correlation matrix* table that provides the mathematical correlation values between the two signals.</div> 

In [ ]:
# No need to edit this code cell
################################
print('correlation matrix:')
p_corr = df_rate.drop('time',axis=1).corr()
# hfig, ax = plt.subplots(1)
# sns.heatmap(p_corr, annot=True)
display(p_corr)

hfig,axs = plt.subplots(1)
axs.scatter(df_rate[channels[0]],df_rate[channels[1]],color = 'black',alpha = 0.1);
axs.set_ylabel('amplitude (a.u.) of your second EMG channel')
axs.set_xlabel('amplitude (a.u.) of your first EMG channel');


## Now let's examine how motor system activity relates to movement kinematics.

<a id="three"></a>
## Part III. Import tracking data from the simultaneously recorded video
### Edit the code cell below with the appropriate information, then play/execute the cell
- **bodyparts** is the list of body parts that are relevant to your analysis of the movement/muscle activity. Each body part must be in quotes. Body parts separated by commas.
- **bones** = is the list of bones that are relevant to your analysis of the movement/muscle activity. Each bone must be in quotes. Bones separated by commas.
<br>

#### Possible body parts: (1 = right, 2 = left)
- wrist1(or 2), elbow1(or 2), shoulder1(or 2), hip1(or 2), knee1(or 2), ankle1(or 2), chin, forehead.

#### Possible bones: 
- refer to the skeleton that you created for your video tracking in Collaboratory notebook. 

<div class="alert alert-success"><b>Task:</b> Running the cell below will import and collect tracking data for the body parts and bones that you specify.</div>

In [ ]:
bodyparts = ['wrist1','shoulder1','shoulder2']
bones = ['wrist1_elbow1']



# No need to edit below this line
################################

df_vid = pd.read_hdf(joints_path) # import data from body parts _filtered.h5
df_vid = df_vid[df_vid.columns[0][0]]
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['_x','_y']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]
# filter by bodyparts
bodyparts = [any(keep in ele for keep in bodyparts) for ele in list(df_vid.columns)]
df_vid_bodyparts = df_vid.loc[:,(np.asarray(coords) & np.asarray(bodyparts))]

df_vid = pd.read_hdf(bones_path) # import data from bones _skeleton.h5
df_vid.columns = df_vid.T.index.map('_'.join)
coords = ['orientation']
# filter by coordinates or liklihood
coords = [any(keep in ele for keep in coords) for ele in list(df_vid.columns)]
# filter by bodyparts
bones = [any(keep in ele for keep in bones) for ele in list(df_vid.columns)]
df_vid_bones = df_vid.loc[:,(np.asarray(coords) & np.asarray(bones))]

df_vid = df_vid_bones.join(df_vid_bodyparts) # combine bones and bodyparts
df_vid =(df_vid - df_vid.mean()) / df_vid.std() # mean subtract and normalize by std
df_vid['time'] = df_vid.index.values / fs_vid # create time column
df_vid = df_vid[((df_vid['time']>start) & (df_vid['time']<stop))] # select start:stop section

print('body parts and bones that you can now analyze for kinematics:')
print(list(df_vid.keys()))
print('')
print('sample of table with positions of body parts and orientations of bones')
display(df_vid.head())



<a id="four"></a>
## Part IV. Calculate some basic movement kinematics - speed
### By editing and running each of the next three code cells (A, B, and C) you can calculate the speed of particular features that you think are important for understanding the movement
> You can do the next 3 code cells in any order. To calculate another kinematic parameter, just change the x, y, or bone inputs and re-run the cell. <br>
<br>
> After running each code cell, the updated data table will be displayed along with a new list of all of the movement parameters that you can analyze.

<div class="alert alert-success"><b>A:</b> Running the cell below gets the absolute speed of a body part (no directional information). Supply the name of the _x and _y components for a specific body part.  Also provide the name you want to call this new calculated parameter.</div>

In [ ]:
x = 'wrist1_x'
y = 'wrist1_y'
new_name = 'wrist1_speed'


# No need to edit below this line
################################
df_vid = get_bodypart_speed_absolute(df_vid,x,y,new_name)
display(df_vid.drop('time',axis=1).head())
print('body parts, bones, and kinematics that you can now examine and compare to muscle activity:')
print(list(df_vid.drop('time',axis=1).keys()))

<div class="alert alert-success"><b>B:</b> Running the cell below gets the speed of a body part in either the x or y direction. Provide the name of a body part appended by either _x or _y. Also provide the name you want to call this new calculated parameter.</div>

In [ ]:
d = 'wrist1_y'
new_name = 'wrist1_yspeed'


# No need to edit below this line
################################
df_vid = get_bodypart_speed(df_vid,d,new_name)
display(df_vid.drop('time',axis=1).head())
print('body parts, bones, and kinematics that you can now examine and compare to muscle activity:')
print(list(df_vid.drop('time',axis=1).keys()))

<div class="alert alert-success"><b>C:</b> Running the cell below gets the speed of a bone (the rate of change of its orientation). Provide the name of a bone (with suffix _orientation). Also provide the name you want to call this new calculated parameter.</div></div>

In [ ]:
bone = 'wrist1_elbow1_orientation'
new_name = 'wrist_elbow_speed'


# No need to edit below this line
################################
df_vid = get_bone_speed(df_vid,bone,new_name)
display(df_vid.drop('time',axis=1).head())
print('body parts, bones, and kinematics that you can now examine and compare to muscle activity:')
print(list(df_vid.drop('time',axis=1).keys()))

<a id="five"></a>
## Part V. Plot bodyparts, bones, or kinematics that you want to visually compare to muscle activity
### Edit the code cell below with the appropriate information, then play/execute the cell
- **kinematics_to_plot** is the list of kinematics or body part positions, or bone orientations that you want to plot in comparison to muscle activity (the plots will all be overlaid)
- **colors_to_plot** = Choose what color you want each kinematic parameter to be plotted as (this will enable you to keep track of which parameter is which in the plot). Do not use "blue" or "orange" (these are the colors that the EMG envelopes will be plotted in).
  - A list of possible colors to use can be found here: https://xkcd.com/color/rgb/

### You can change these lists and re-plot to look at different variables.


<div class="alert alert-success"><b>Task:</b> After editing the 'kinematics_to_plot' and 'colors_to_plot' variables, run the cell below to overlay all of these variables on the same plot.</div>

In [ ]:
kinematics_to_plot = ['wrist_elbow_speed','wrist1_speed','wrist1_yspeed']
colors_to_plot = ['green','aqua','lime']


# No need to edit below this line
################################
hfig,ax = plt.subplots(1)
for k,c in zip(kinematics_to_plot,colors_to_plot):
    ax.plot(df_vid['time'],df_vid[k],linewidth = 2,color = sns.xkcd_rgb[c],label = k,alpha = 0.75)
ax.plot(df_rate['time'],df_rate[channels[0]].values,linewidth = 2,color = sns.xkcd_rgb['blue'],alpha = 0.75,label = 'EMG channel' + str(channels[0]))
ax.plot(df_rate['time'],df_rate[channels[1]].values,linewidth = 2,color = sns.xkcd_rgb['orange'],alpha = 0.75,label = 'EMG channel' + str(channels[1]))
plt.legend();

### Think about how you would qualitatively describe the relationship between the two muscle signals and the kinematic variables (and the relationship among kinematic variables). As part of your answer, include statements about correlation. 

<a id="six"></a>
## Part VI. You might be thinking... ***THIS IS A LOT TO KEEP TRACK OF!***
### In this section, we will start to implement "*Dimensionality Reduction*" - a technique commonly used in neuroscience to make sense of "BIG DATA."
> You will read and learn more about this in the paper from Mark Churchlan's lab. But let's start to get a feel for it.

<div class="alert alert-success"><b>Task:</b> Run the cell below to perform a "Principal Components Analysis" (PCA) on the ALL of the tracking and kinematics data that you have used to quantif the movement.</div>

In [ ]:
# No need to edit this code cell the first time you run in
################################
df = df_vid.drop('time',axis=1)
df =(df - df.mean()) / df.std()
n_dim = 10
n_components=np.min([n_dim,df.shape[1]]) # if try to take more components than have channels, use amount of channels
print('You quantified your movement into ' + str(len(df.columns)) + ' positional and kinematic variables')
pca = PCA(n_components=n_components)
pca.fit(df)
df_pca = pd.DataFrame(pca.transform(df), columns=['PC%i' % i for i in range(n_components)], index=df.index)
print('Your movement has now been transformed into ' + str(n_components) + ' principle components.')

<div class="alert alert-success"><b>Task:</b> Run the cell below to overlay the first two principle components (orthogonal dimensions) of the result. No need to edit the cell.</div>

In [ ]:
hfig,ax = plt.subplots(1)
ax.set_xlabel('seconds')
ax.set_ylabel('PC units')
ax.set_yticklabels([])
ax.plot(df_vid['time'],df_pca['PC0'],label = 'PC0',alpha = 0.75)
ax.plot(df_vid['time'],df_pca['PC1'],label = 'PC1',alpha = 0.75)
plt.legend();

## PC0 is the first principal component of the movement, PC1 is the second, etc.
> Importantly, these principal components are *independent* of each other. They are *orthogonal* dimensions of the movement.
### Go back and edit the last cell (the one that made a plot of PC0 versus PC1) so that you can also see a plot of PC2. 
- find the line that says ```ax.plot(df_vid['time'],df_pca['PC0'],label = 'PC0')```
- copy that line (ctrl + C on the keyboard), make a new line, and paste it (ctrl + V on the keyboard).
- change 'PC0' to 'PC2' everywhere in your pasted line that you see it. 
- re-run the code cell and see how your plot changed! <br>

### OK, ONE MORE! This time, go back and edit the last cell again so that you can also see a plot of the last principal component.   
- find the line that you just added and edited. It should currently say ```ax.plot(df_vid['time'],df_pca['PC2'],label = 'PC2', alpha = 0.75)```
- go back to the output of the last code cell and see how many principle components were used to describe your movement. 
- change 'PC2' to the last principle component (this will be the number of total principle components minus 1). 
- re-run the code cell and see how your plot changed!

## What's going on here? 
> Think about some of the differences that you notice between the first couple of principle components and the last couple of principle components. The following plot should quantitatively correlate with some of the things that you are qualitatively noticing. 
<div class="alert alert-success"><b>Task:</b> Run the cell below to see a scatter plot showing how much of the variance in the data is explained in each principal component.</div>

In [ ]:
# No need to edit this code cell
################################
hfig,ax = plt.subplots(1)
ax.scatter(df_pca.columns,pca.explained_variance_ratio_,color='black',s=100)
ax.set_ylabel('Explained Variance')
ax.set_xlabel('Principal Components')
ax.set_ylim(-0.05,1.05)

## Dimensionality reduction techniques, such as principle components analysis (PCA), enable us to represent and quantify high-dimensional things (like neural populations and movements) in a new *space*. 
> These PCA spaces are in some ways abstract. But in other ways, the abstract space is actually the essence of the neural activity or movement. 
### What is the "shape" of this movement in the space defined by the PCA dimensions?
<div class="alert alert-success"><b>Task:</b> Running the cell below plots the first principal component against the second.</div>

In [ ]:
# No need to edit this code cell
################################
fig, ax = plt.subplots(1)
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.plot(df_pca['PC0'],df_pca['PC1'],color = 'black',alpha = 0.75,linewidth=2);

### But how does the movement evolve over time? (ie. where is time in the plot?)
<div class="alert alert-success"><b>Task:</b> Running the cell below animates the movement over time through this "latent" space - the space defined by the principal components.</div>

In [ ]:
# No need to edit this code cell
################################

fig, ax = plt.subplots(1)
ax.plot(df_pca['PC0'],df_pca['PC1'],color = 'black',alpha = 0.5,linewidth=2)
t, = ax.plot(df_pca['PC0'].values[0], df_pca['PC1'].values[0],color = 'red',marker='.',markersize = 20)

def animate(i):
    ax.plot(df_pca['PC0'].values[i], df_pca['PC1'].values[i],color = 'red',marker='.',markersize = 15,zorder=3,alpha = 0.25)
    
ani = animation.FuncAnimation(
    fig, animate, frames = len(df_pca),interval=10, repeat = False)
plt.show()

### And how does the movement look in the 3-dimensional space defined by the PCA? 
<div class="alert alert-success"><b>Task:</b> Running the cell below plots the movement in the first three principle components. You can rotate the plot around in 3D.</div>

In [ ]:
# No need to edit this code cell
################################
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(df_pca['PC0'],df_pca['PC1'],df_pca['PC2'],color = 'black',alpha = 0.5,linewidth=2)
ax.set_xlabel('PC0')
ax.set_ylabel('PC1')
ax.set_zlabel('PC2')

### And how does the movement in PCA space compare to the muscle activity in muscle space? 
<div class="alert alert-success"><b>Task:</b> Running the cell below plots the activity of muscle 1 against muscle 2.</div>

In [ ]:
# No need to edit this code cell
################################
fig, ax = plt.subplots(1)
ax.set_xlabel('muscle activity 1')
ax.set_ylabel('muscle activity 2')
ax.plot(df_rate[channels[0]],df_rate[channels[1]],color = 'black',alpha = 0.75,linewidth=2);

# Take-home assignmnent

## Review: Why/What dimensionality reduction?
"Describing a complex signal, such as a visual scene or a pattern of neural activity, in terms of just a few summarizing features is called dimensionality reduction" (Pang et al 2016). This is a tool that has successfully been used to achieve one of the main goals in neuroscience -- to objectively quantify what the brain represents and computes. BUT this tool can be daunting, 'mathy,' and abstract when first encountered. This module and its analyses are designed to help you scaffold your intuition for what dimensionality reduction does and its power as an analytic tool to generate representations of stimuli, movements, and brain activity that are concise, complete, and informative about the workings of the nervous system, without being biased by arbitrary experimental choices. Dimensionality reduction (and specifically Principle Component Analysis introduced in this lab) provides an entry point into principled mathematical techniques that let us discover these representations directly from experimental data, a key step to developing rich yet comprehensible models for brain function.

I like the following example from Pang et al (2016) "Dimensionality reduction in neuroscience."
"An example of dimensionality reduction is color vision. Light hitting the eye has intensity in a wide range of frequencies. While a spectrophotometer would provide a complete description of the light beam in terms of its power spectrum across all frequencies, our retina has only three kinds of color sensor, the L, M and S cone types (corresponding to long, medium, and short wavelengths). All we can know about the incoming light is given to us by the activation of those three sensors: because of the unique frequency absorption properties of each cone type, the activation of a given cone type is a function of a weighted sum of the light’s intensities at different frequencies. Thus, our color perception is a three-dimensional representation of the original, infinite-dimensional spectrogram that specifies the light’s intensity at every frequency."

## Questions to answer: Use the prompts and questions to create the following 6 figures. Use a separate document to answer the following questions in a separate document (Fig 2, 5 and 6 do not need separate text). Answer the questions in the format of a concise paragraph (rather than a bullet point list of answers). Consolidate, summarize, interpret. Refer to panels in the figures you create (for example as Figure 1A) in order to visualize/show what you write in words.

### Figure 1. *Correlations*
Pick one experimental condition.
- Based on the data that you recorded (not the movement that you know you did), describe the pattern of muscle activity that you observed.  
- Interpret the correlation value between the activity in the two muscles you recorded. 

### Figure 2. *Movement Tracking Results*
Use the same experimental condition as in Figure 1.
- Take a screenshot of your labelled video. 
- Annotate the body parts and bones that you analyzed in order to relate movement to muscle activity.
- Label the muscles that you recorded (and placement of electrodes)

### Figure 3. *Dimensionality Reduction*
Use the same experimental condition as in Figure 1.
- Describe the relationship between the movement kinematics you calculated and the muscle activity. Summarize patterns and categorize similar variables together rather than describing every pairwise comparison explicitly. 
- How would you describe the way the movement looks in PCA space? In theory, think about whether you could use kinematic variables to make a plot that looks similar. If you did this, how many kinematic variables would you be able to incorporate in your plot?
- Describe the relationship between muscle activity and the first several PCAs (however many are useful based on your analysis). 
- What is the difference between low number PCAs and high number PCAs? How would you describe the reason for this trend to someone who has even less understanding of PCA than you do?

### Figure 4. *Data interpretation*
Analyze both of your experimental conditions to compare.
- Compare the movement in PCA space for your two different conditions.
- Compare the muscle activity for your two different conditions.
- How did the relationship between muscle activity and movement change? Why? You can use what you learned from the papers that we read for this module as background to help you describe. 
- Figure 2 from Russo et al (2018) shows a dissociation between movement kinematics and muscle activity. How does Figure 2 show this? Did your task comparison achieve this?

### Figure 5. *Connections to Primary Literature*
- Examine Figure 1 of the Russo et al (2018) paper and answer the following. Annotate the Figure to identify the following elements. Any additional information can be included in the legend. 
  - Where is the trial-averaged neural activity?
  - What is the onset of a trial in this experiment?
  - Where is the baseline activity of the neuron depicted?
  - Where is an example individual trial of neural activity depicted? 
  - Panel H is "rectified and smoothed" muscle activity while F shows raw muscle activity (but zoomed out). Based on the average in H and the raw EMG data that you recorded and visualized, draw what you think the raw EMG voltage trace from one trial looks like for the example muscle shown.

### Figure 6. *Connections to Primary Literature*
- Recreate Figure 4, Panels A, B, C, and D from your data.
- Use the legend from the paper as a template to help you write your legend.